## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in Toronto City</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>

## 1. Download and Explore Dataset

#### Load and explore the data

let's load the data.

In [3]:
df_to = pd.read_excel("toronto-FSAs.xlsx",header=0,names=["Postcode","Borough","Neighbourhood"])

Let's take a quick look at the data.

we remove the record with Borough is 'Not assigned' 

In [5]:
df_to1 = df_to[df_to["Borough"] != "Not assigned "]

Let's take a look at the first 10 items in this dataframe.

In [6]:
df_to1.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


let's group the dataframe with sevral neighbourhoods under one postcode 

In [7]:
df_to2 = df_to1.groupby(['Postcode','Borough'], as_index=False, sort=False).agg(','.join)
df_to2.replace('Not assigned', 'Queen\'s Park', inplace = True)
df_to2.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


#### add Latitude and Longitude to the dataframe

we load the csv with postcode's latitude and longtitude

In [8]:
df_geo = pd.read_csv("Geospatial_Coordinates.csv",header=0,names=["Postcode","Latitude","Longitude"])

let's have a look with it

In [9]:
df_geo.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let's merge this tow dataframe to one.

In [10]:
df_to2['Postcode']=df_geo['Postcode'].astype('str')
df_geo['Postcode']=df_geo['Postcode'].astype('str')
df_to_geo=pd.merge(df_to2,df_geo,on = 'Postcode')

Quickly examine the resulting dataframe.

In [11]:
df_to_geo.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,North York,Parkwoods,43.806686,-79.194353
1,M1C,North York,Victoria Village,43.784535,-79.160497
2,M1E,Downtown Toronto,"Harbourfront,Regent Park",43.763573,-79.188711
3,M1G,North York,"Lawrence Heights,Lawrence Manor",43.770992,-79.216917
4,M1H,Queen's Park,Queen's Park,43.773136,-79.239476


let's have a look with its shape

In [12]:
df_to_geo.shape

(103, 5)

#### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>ny_explorer</em>, as shown below.

In [14]:
latitude = 43.6532
longitude= -79.3832

#### Create a map of New York with neighborhoods superimposed on top.

In [18]:
# create map of New York using latitude and longitude values
map_to = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_to_geo['Latitude'], df_to_geo['Longitude'], df_to_geo['Borough'], df_to_geo['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to